# Part 1.7. Tips

## 1. Maximum Likelihood Estimation
모집단의 분포 함수의 대략적인 형태는 알지만 어떤 정보를 모른다고 할 때, 해당 분포함수를 다음과 같이 나타날 수 있다. 이 때 $\theta$를 내가 모르는 어떤 정보라 하자.

$$f(x;\theta)$$

샘플 $x_1, x_2, ..., x_n$을 얻었다고 할 때 이 샘플의 분포함수는 다음과 같이 나타날 수 있으며 이를 **가능도 함수**라고 한다. 이 샘플를 얻은 것은 우연이 아니라 필연이라고 생각한다.

$$L(\theta;x_1, ... ,x_n) = \prod_{i=1}^nf(x_i;\theta)$$

이 때 가능도 함수가 최대이도록 하는 $\theta$를 우리가 원하는 추청값이다.

$$L(\hat{\theta}^{MLE};x_1, ..., x_n) = \max_\theta L(\theta;x)$$ 

## 2. Optimization via Gradient Descent
그래서 우리는 이 $\theta$를 경사하강법(Gradient Descent)를 통해서 찾게 된다. 정확히 말하면 위의 가능도 함수에 대한 $\theta$의 기울기를 구하여 가능도 함수가 최대가 되도록 $\theta$를 업데이트 시킨다.

$$\theta \leftarrow \theta - \alpha \triangledown_\theta L(x;\theta)$$

## 3. Overfitting and Regularization
### 3.1. Overfitting
오버피팅이란 주어진 데이터에 대해서 과도하게 맞추어졌을 때, 피팅이 되었을 때의 상황을 말한다. MLE의 경우 해당 데이터를 가장 잘 설명하는 확률 분포 함수를 찾다 보니 오버피팅이 일어날 수 밖에 없다.

![7-1.png](./img/7-1.png)

### 3.2. How to Prevent Overfitting
#### 3.2.1. Training, Validation, Test Dataset
이 오버피팅을 막기 위해 데이터셋은 **훈련 데이터셋(Train Dataset)**, **검증 데이터셋(Validation Dataset)**, **테스트 데이터셋(Test Dataset)**으로 나누게 된다. 테스트 데이터셋은 훈련 데이터셋과 비슷할 것이라 가정한다. 이 테스트 데이터셋을 통해서 해당 모델의 성능을 평가할 수 있다.

더 나아가 테스트 데이터셋도 오버피팅이 되는 경우가 있는데 그걸 방지하기 위해 검증 데이터셋을 테스트 데이터 전에 한 번 검증을 하는 것이다. 이로 모델의 성능을 더 높일 수 있다.

![7-2.png](./img/7-2.png)

#### 3.2.2. Regularization
* **early stopping** : validation loss가 더이상 낮아지지 않을 때 학습을 멈추는 것
* **reducing network size** : 신경망의 사이즈를 줄여 학습할 수 있는 양을 줄이는 방법
* **weight decay** : 신경망에 쓰이는 매개변수의 개수를 제한
* **dropout**과 **batch normalization**이 가장 많이 쓰인다.

#### 3.2.3. Other Ways
더 많은 데이터를 모으거나 쓸모 없는 피처를 없애는 방법도 있다.

## 4. Basic Approach to Train DNN
1. 신경망 구조를 설계한다.
2. 학습을 하고 모델이 오버피팅이 되었는지 체크한다.
  * 그렇지 않다면 모델을 더 깊고 넓게 확장한다.
  * 그렇다면 drop out이나 batch normalization과 같은 regularization을 한다.
3. 2.를 반복한다.

## 5. Implementation

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### 5.1. Training and Test Datasets

In [0]:
x_train = torch.FloatTensor([[1, 2, 1],                     # (m, 3)
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])        # (m, )

In [0]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])   # (m', 3)
y_test = torch.LongTensor([2, 2, 2])                            # (m', )

### 5.2. Model

In [0]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 3)   # 피처 3개, 라벨 3개
  def forward(self, x):
    return self.linear(x)

In [0]:
model = SoftmaxClassifierModel()

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

### 5.3. Training

In [0]:
def train(model, optimizer, x_train, y_train):
  nb_epochs = 20
  for epoch in range(nb_epochs):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print("Epoch {:4d}/{} Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

In [0]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]    # (최대값, 최대값의 인덱스)

    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(correct_count / len(y_test) * 100, cost.item()))

In [14]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [15]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


## 6. Learning Rate
학습 속도를 조절할 수 있는 하이퍼파라미터이다. 학습률이 너무 크면 발산하면서 cost가 점점 늘어날 수 있다. 반대로 학습률이 너무 작으면 cost가 너무 줄어들지 않는다. 그러니 발산하면 줄이고, 줄어들지 않으면 키우면서 적절한 값을 찾는 것이 중요!

In [17]:
# learning rate가 클 때
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e5)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 1100707.375000
Epoch    2/20 Cost: 2482261.000000
Epoch    3/20 Cost: 664769.875000
Epoch    4/20 Cost: 1668198.875000
Epoch    5/20 Cost: 748657.812500
Epoch    6/20 Cost: 1353832.375000
Epoch    7/20 Cost: 1790073.750000
Epoch    8/20 Cost: 917894.875000
Epoch    9/20 Cost: 989687.125000
Epoch   10/20 Cost: 990845.250000
Epoch   11/20 Cost: 1585082.500000
Epoch   12/20 Cost: 1265073.750000
Epoch   13/20 Cost: 1149145.000000
Epoch   14/20 Cost: 589766.937500
Epoch   15/20 Cost: 689678.625000
Epoch   16/20 Cost: 983032.687500
Epoch   17/20 Cost: 1265073.750000
Epoch   18/20 Cost: 1686645.000000
Epoch   19/20 Cost: 484999.593750


In [18]:
# learning rate가 작을 때
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e-10)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.341574
Epoch    2/20 Cost: 1.341574
Epoch    3/20 Cost: 1.341574
Epoch    4/20 Cost: 1.341574
Epoch    5/20 Cost: 1.341574
Epoch    6/20 Cost: 1.341574
Epoch    7/20 Cost: 1.341574
Epoch    8/20 Cost: 1.341574
Epoch    9/20 Cost: 1.341574
Epoch   10/20 Cost: 1.341574
Epoch   11/20 Cost: 1.341574
Epoch   12/20 Cost: 1.341574
Epoch   13/20 Cost: 1.341574
Epoch   14/20 Cost: 1.341574
Epoch   15/20 Cost: 1.341574
Epoch   16/20 Cost: 1.341574
Epoch   17/20 Cost: 1.341574
Epoch   18/20 Cost: 1.341574
Epoch   19/20 Cost: 1.341574


## 7. Data Preprocessing
머신이 학습하기 쉽도록 데이터를 미리 전처리하는 과정을 말한다. 예를 들어 한 피처의 값의 범위가 1000단위이고 다른 한 범위가 소수점 단위라고 할 때 신경망은 1000단위의 값을 조정하는 게 더 이득을 보기에 해당 피처를 가지고만 학습을 할 것이다. 이를 방지하기 위해 전처리를 해주어 동등하게 학습하도록 해주어야 한다.

![7-3.png](./img/7-3.png)

### 7.1. Standardization
데이터의 수치를 정규 분포로 바꿔주는 방법. 이 때 우리가 가진 데이터가 정규 분포를 따른다고 가정하고 평균($\mu$)과 표준편차($\sigma$)를 이용해 정규화시켜준다.

$$x'_j = \frac{x_j-\mu_j}{\sigma_j}$$

In [0]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [20]:
mu = x_train.mean(dim=0)                  # 평균
sigma = x_train.std(dim=0)                # 표준편차
norm_x_train = (x_train - mu) / sigma     # 정규화된 데이터

print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [0]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)   # 3개의 피처로 특정 수치를 예측

    def forward(self, x):
        return self.linear(x)

In [0]:
model = MultivariateLinearRegressionModel()

In [0]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [0]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [25]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29785.089844
Epoch    1/20 Cost: 18906.164062
Epoch    2/20 Cost: 12054.674805
Epoch    3/20 Cost: 7702.029785
Epoch    4/20 Cost: 4925.733398
Epoch    5/20 Cost: 3151.632812
Epoch    6/20 Cost: 2016.996094
Epoch    7/20 Cost: 1291.051270
Epoch    8/20 Cost: 826.505249
Epoch    9/20 Cost: 529.207397
Epoch   10/20 Cost: 338.934204
Epoch   11/20 Cost: 217.153564
Epoch   12/20 Cost: 139.206757
Epoch   13/20 Cost: 89.313782
Epoch   14/20 Cost: 57.375465
Epoch   15/20 Cost: 36.928429
Epoch   16/20 Cost: 23.835773
Epoch   17/20 Cost: 15.450401
Epoch   18/20 Cost: 10.077809
Epoch   19/20 Cost: 6.633700
